In [1]:
from transform_csv import rename_string, check_long_lat, transform_long_lat
import numpy as np
import pandas as pd

In [2]:

df_listings = pd.read_csv("inside_airbnb_la/listings/listings_detailed.csv")
df_reviews = pd.read_csv("inside_airbnb_la/reviews/reviews_detailed.csv")

In [3]:
# Check and process column names
df_listings = df_listings.rename(columns=rename_string)
df_listings.apply(lambda x: check_long_lat(x["longitude"], x["latitude"]), axis=1)
df_listings["location"] = df_listings.apply(lambda x: transform_long_lat(x["longitude"], x["latitude"]), axis=1)
df_listings = df_listings.drop(columns=["longitude", "latitude"])

In [4]:
print(df_listings.columns)
print(df_listings["id"])

Index(['id', 'listing_url', 'scrape_id', 'last_scraped', 'source', 'name',
       'description', 'neighborhood_overview', 'picture_url', 'host_id',
       'host_url', 'host_name', 'host_since', 'host_location', 'host_about',
       'host_response_time', 'host_response_rate', 'host_acceptance_rate',
       'host_is_superhost', 'host_thumbnail_url', 'host_picture_url',
       'host_neighbourhood', 'host_listings_count',
       'host_total_listings_count', 'host_verifications',
       'host_has_profile_pic', 'host_identity_verified', 'neighbourhood',
       'neighbourhood_cleansed', 'neighbourhood_group_cleansed',
       'property_type', 'room_type', 'accommodates', 'bathrooms',
       'bathrooms_text', 'bedrooms', 'beds', 'amenities', 'price',
       'minimum_nights', 'maximum_nights', 'minimum_minimum_nights',
       'maximum_minimum_nights', 'minimum_maximum_nights',
       'maximum_maximum_nights', 'minimum_nights_avg_ntm',
       'maximum_nights_avg_ntm', 'calendar_updated', 'has_ava

In [5]:
# df_reviews.listing_id is a foreign key to df_listings.id
print(df_reviews["listing_id"].unique().shape)

(32948,)


In [6]:
df_reviews.head()

,listing_id,id,date,reviewer_id,reviewer_name,comments
0,109,449036,2011-08-15,927861,Edwin,The host canceled my reservation the day befor...
1,109,74506539,2016-05-15,22509885,Jenn,Me and two friends stayed for four and a half ...
2,2708,13994902,2014-06-09,10905424,Kuberan,i had a wonderful stay. Everything from start ...
3,2708,14606598,2014-06-23,2247288,Camilla,Charles is just amazing and he made my stay sp...
4,2708,39597339,2015-07-25,27974696,Fallon,Staying with Chas was an absolute pleasure. He...


In [7]:
# convert review into json to match mongo document structure
# "reviews": [
#         {
#             "_id": "58663741",
#             "date": {
#                 "$date": "2016-01-03T05:00:00.000Z"
#             },
#             "listing_id": "10006546",
#             "reviewer_id": "51483096",
#             "reviewer_name": "Cátia",
#             "comments": "A casa da Ana e do Gonçalo foram o local escolhido para a passagem de ano com um grupo de amigos. Fomos super bem recebidos com uma grande simpatia e predisposição a ajudar com qualquer coisa que fosse necessário.\r\nA casa era ainda melhor do que parecia nas fotos, totalmente equipada, com mantas, aquecedor e tudo o que pudessemos precisar.\r\nA localização não podia ser melhor! Não há melhor do que acordar de manhã e ao virar da esquina estar a ribeira do Porto."
#         },
#         ]


def mongolize_date(date):
    """
    Convert the date string from YYYY-MM-DD to a ISO 8601, which mongoimport can recognize
    """
    return date + "T00:00:00.000Z"


def mongolize_reviews(review):
    return {
        "date": {
            "$date": mongolize_date(review["date"])
        },
        "listing_id": review["listing_id"],
        "reviewer_id": review["reviewer_id"],
        "reviewer_name": review["reviewer_name"],
        "comments": review["comments"]
    }


df_reviews["reviews"] = df_reviews.apply(lambda x: mongolize_reviews(x), axis=1)

In [8]:
df_reviews["reviews"][0]

{'date': {'$date': '2011-08-15T00:00:00.000Z'},
 'listing_id': 109,
 'reviewer_id': 927861,
 'reviewer_name': 'Edwin',
 'comments': 'The host canceled my reservation the day before arrival.'}

In [9]:
# Collect reviews of the same listing_id into one array
df_reviews_grouped = df_reviews.groupby("listing_id")["reviews"].apply(list).reset_index()

In [10]:
print(df_reviews_grouped.shape)
df_reviews_grouped.head()

(32948, 2)


,listing_id,reviews
0,109,[{'date': {'$date': '2011-08-15T00:00:00.000Z'...
1,2708,[{'date': {'$date': '2014-06-09T00:00:00.000Z'...
2,2732,[{'date': {'$date': '2011-06-06T00:00:00.000Z'...
3,5728,[{'date': {'$date': '2009-07-17T00:00:00.000Z'...
4,5843,[{'date': {'$date': '2011-05-17T00:00:00.000Z'...


In [11]:
# Join df_reviews_grouped with df_listings on listing_id
df_listings_and_reviews = pd.merge(df_listings, df_reviews_grouped, left_on="id", right_on="listing_id", how="left")

In [12]:
print(df_listings_and_reviews.shape)
print(df_listings.shape)

(45591, 76)
(45591, 74)


In [13]:
df_listings_and_reviews["reviews"][0]

[{'date': {'$date': '2011-08-15T00:00:00.000Z'},
  'listing_id': 109,
  'reviewer_id': 927861,
  'reviewer_name': 'Edwin',
  'comments': 'The host canceled my reservation the day before arrival.'},
 {'date': {'$date': '2016-05-15T00:00:00.000Z'},
  'listing_id': 109,
  'reviewer_id': 22509885,
  'reviewer_name': 'Jenn',
  'comments': "Me and two friends stayed for four and a half months. It was a great place to stay! The apartment was very comfortable and I really enjoyed having the park with running path across the street. The only downside was it wasn't within walking distance to restaurants, bars, or coffee shops. But they are a short drive away. Overall, great stay!"}]

In [14]:
# Drop unnecessary columns
useless_cols = ["host_neighbourhood",
                "host_listings_count",
                "host_total_listings_count",
                "host_has_profile_pic",
                "availability_30",
                "availability_60",
                "availability_90",
                "availability_365",
                "instant_bookable",
                "host_thumbnail_url",
                "host_picture_url",
]

df_listings_and_reviews = df_listings_and_reviews.drop(columns=useless_cols)

In [15]:
df_listings_and_reviews.to_json("listing_and_reviews.json", orient="records", lines=True)